In [8]:
%load_ext autoreload
%autoreload 2

from tweedejaars_project import *
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
features = [
    'import_capacity',
    # 'min_price_published',
    'mid_price_published',
    # 'max_price_published',
    # 'min_ptu_price_known',
    # 'max_ptu_price_known',
    # 'settlement_price_bestguess',
    'time_since_last_two_sided',
    'two_sided_daily_count',
    'PTU',
    'naive_strategy_action',
    'forecast_wind',
    'forecast_solar',
    'forecast_demand',
    'ptu_id',
    'fix_two_sided_ptu_realtime'
]

# already used
target = 'target_two_sided_ptu'

In [10]:
df = load_df()
splits = get_splits(df, features)

In [11]:
# Hyperparameters
ptu_window = 5 # Dit is het PTU aantal dus ptu_id's niet row aantal
batch_size = 15000
input_size = len(features)
hidden_size = 16
num_layers = 2
output_size = 1
num_epochs = 1
learning_rate = 0.001


# Split the data in vars
train_data = splits['train']
# test_data = splits['test']

rows = 20
print(train_data['in']['ptu_id'])


0            0
1            0
2            0
3            0
4            0
          ... 
123835    8255
123836    8255
123837    8255
123838    8255
123839    8255
Name: ptu_id, Length: 123840, dtype: int64


In [325]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, ptu_window, ptu_length=15):
        self.data_in = pd.DataFrame(data['in']).astype(np.float32)
        self.data_out = pd.Series(data['out']).astype(np.float32)
        self.ptu_window = ptu_window
        self.ptu_length = ptu_length

        self.ptu_history = self.ptu_window * self.ptu_length  # Entire window

        self.sequence_indices, self.sequence_lengths = self.create_sequences()

    def create_sequences(self):
        sequence_indices, sequence_lengths = [], []

        row_idx = 1  # Index of the current row
        start_idx = 0  # Index of the furthest row in history

        counter = self.ptu_length

        for _ in range(len(self.data_in)):
            if row_idx > self.ptu_history:  # Start using the counter only after padding is passed
                counter -= 1
                if counter == 0:
                    start_idx += self.ptu_length
                    counter = self.ptu_length

            sequence_indices.append((start_idx, row_idx))
            sequence_lengths.append(row_idx - start_idx)  # Add length of sequence

            row_idx += 1


        return sequence_indices, sequence_lengths

    def __len__(self):
        return len(self.sequence_indices)

    def __getitem__(self, idx):
        start_idx, row_idx = self.sequence_indices[idx]
        length = self.sequence_lengths[idx]

        # np array sequence and target 
        sequence = self.data_in.iloc[start_idx:row_idx].values  
        target = self.data_out.iloc[row_idx]

        sequence = torch.tensor(sequence, dtype=torch.float32).flip(0)  # draai het om

        # Dit wordt gebruikt voor padding alleen voor de eerdere getallen  waar de history nog incompleet is. 
        if length < self.ptu_history:
            # print(length, start_idx, row_idx)
            sequence = F.pad(sequence, (0, 0, 0, (self.ptu_history - length) + self.ptu_length), mode='constant', value=np.nan)
        
        else:
            # Padding for the dynamic rows in the current ptu
            current_seq_len = self.ptu_length - ((row_idx - start_idx) % self.ptu_length)
            sequence = F.pad(sequence, (0, 0, 0, current_seq_len), mode='constant', value=np.nan)  # Extra padding
        
        # print(sequence.shape)

        return sequence, torch.tensor(target, dtype=torch.float32), length


In [326]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, lengths):
        packed_input = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.rnn(packed_input)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        
        # Use the last valid output for each sequence
        idx = (lengths - 1).view(-1, 1).expand(output.size(0), output.size(2)).unsqueeze(1)
        output = output.gather(1, idx).squeeze(1)
        
        output = self.fc(output)
        return output


In [327]:
def prepare_data_loader(data, ptu_window, batch_size):
    dataset = TimeSeriesDataset(data, ptu_window)
    # all_sequences, all_targets = dataset.get_all_sequences()
    print('done')
    return DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [328]:
train_loader = prepare_data_loader(train_data, ptu_window, batch_size)


done


In [329]:
# TODO: eerste batch eerste item is puur nans?
# TODO: Special NaN value for all inputs of NaN
train_features, train_labels, lenghts = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
print(f'lenghts: {lenghts}')


Feature batch shape: torch.Size([15000, 90, 11])
Labels batch shape: torch.Size([15000])
lenghts: tensor([ 1,  2,  3,  ..., 88, 89, 75])


In [314]:
x = 89
feature_tensor = train_features[x]
lenght = lenghts[x].item()
feature_tensor, lenght

(tensor([[7.0950e+02, 3.9390e+01, 5.0000e+00, 1.0000e+00, 1.0000e+01, 1.0000e+00,
          1.0059e+04, 0.0000e+00, 1.0741e+04, 5.0000e+00, 0.0000e+00],
         [7.0950e+02, 3.9390e+01, 4.9333e+00, 1.0000e+00, 1.0000e+01, 1.0000e+00,
          1.0059e+04, 0.0000e+00, 1.0741e+04, 5.0000e+00, 0.0000e+00],
         [7.0950e+02, 3.9390e+01, 4.8667e+00, 1.0000e+00, 1.0000e+01, 1.0000e+00,
          1.0059e+04, 0.0000e+00, 1.0741e+04, 5.0000e+00, 0.0000e+00],
         [7.0950e+02, 3.9390e+01, 4.8000e+00, 1.0000e+00, 1.0000e+01, 1.0000e+00,
          1.0059e+04, 0.0000e+00, 1.0741e+04, 5.0000e+00, 0.0000e+00],
         [7.0950e+02, 3.9390e+01, 4.7333e+00, 1.0000e+00, 1.0000e+01, 0.0000e+00,
          1.0059e+04, 0.0000e+00, 1.0741e+04, 5.0000e+00, 0.0000e+00],
         [7.0950e+02, 3.9390e+01, 4.6667e+00, 1.0000e+00, 1.0000e+01, 0.0000e+00,
          1.0059e+04, 0.0000e+00, 1.0741e+04, 5.0000e+00, 0.0000e+00],
         [7.0950e+02, 3.9390e+01, 4.6000e+00, 1.0000e+00, 1.0000e+01, 0.0000e+00,


In [75]:
for batch_idx, (sequences, targets) in enumerate(train_loader):
    print("Batch:", batch_idx)
    print("Sequence shape:", sequences.shape)  # Print the shape of the input sequences
    print("Targets shape:", targets.shape)  # Print the shape of the targets


torch.Size([1, 11])
torch.Size([2, 11])
torch.Size([3, 11])
torch.Size([4, 11])
torch.Size([5, 11])
torch.Size([6, 11])
torch.Size([7, 11])
torch.Size([8, 11])
torch.Size([9, 11])
torch.Size([10, 11])
torch.Size([11, 11])
torch.Size([12, 11])
torch.Size([13, 11])
torch.Size([14, 11])
torch.Size([15, 11])
torch.Size([16, 11])
torch.Size([17, 11])
torch.Size([18, 11])
torch.Size([19, 11])
torch.Size([20, 11])
torch.Size([21, 11])
torch.Size([22, 11])
torch.Size([23, 11])
torch.Size([24, 11])
torch.Size([25, 11])
torch.Size([26, 11])
torch.Size([27, 11])
torch.Size([28, 11])
torch.Size([29, 11])
torch.Size([30, 11])
torch.Size([31, 11])
torch.Size([32, 11])
torch.Size([33, 11])
torch.Size([34, 11])
torch.Size([35, 11])
torch.Size([36, 11])
torch.Size([37, 11])
torch.Size([38, 11])
torch.Size([39, 11])
torch.Size([40, 11])
torch.Size([41, 11])
torch.Size([42, 11])
torch.Size([43, 11])
torch.Size([44, 11])
torch.Size([45, 11])
torch.Size([46, 11])
torch.Size([47, 11])
torch.Size([48, 11])
t

KeyboardInterrupt: 

In [21]:
def train_rnn(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        for batch_idx, (sequences, targets) in enumerate(train_loader):
            # Prepare the data
            sequences = sequences.float()
            targets = targets.float()
            lengths = torch.tensor([len(seq) for seq in sequences])
            
            # Debugging print statements
            print(f"Batch {batch_idx}, Sequence shape: {sequences.shape}, Lengths: {lengths}")
            
            # Forward pass
            outputs = model(sequences, lengths)
            
            # Compute loss
            targets = targets.unsqueeze(1)  # Ensure targets have the shape (batch_size, 1)
            loss = criterion(outputs, targets)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (batch_idx + 1) % 10 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


model = RNNModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_rnn(model, train_loader, criterion, optimizer, num_epochs)


Batch 0, Sequence shape: torch.Size([15000, 90, 11]), Lengths: tensor([90, 90, 90,  ..., 90, 90, 90])
Batch 1, Sequence shape: torch.Size([15000, 90, 11]), Lengths: tensor([90, 90, 90,  ..., 90, 90, 90])
Batch 2, Sequence shape: torch.Size([15000, 90, 11]), Lengths: tensor([90, 90, 90,  ..., 90, 90, 90])
Batch 3, Sequence shape: torch.Size([15000, 90, 11]), Lengths: tensor([90, 90, 90,  ..., 90, 90, 90])
Batch 4, Sequence shape: torch.Size([15000, 90, 11]), Lengths: tensor([90, 90, 90,  ..., 90, 90, 90])
Batch 5, Sequence shape: torch.Size([15000, 90, 11]), Lengths: tensor([90, 90, 90,  ..., 90, 90, 90])
Batch 6, Sequence shape: torch.Size([15000, 90, 11]), Lengths: tensor([90, 90, 90,  ..., 90, 90, 90])
Batch 7, Sequence shape: torch.Size([15000, 90, 11]), Lengths: tensor([90, 90, 90,  ..., 90, 90, 90])
Batch 8, Sequence shape: torch.Size([3840, 90, 11]), Lengths: tensor([90, 90, 90,  ..., 90, 90, 90])


In [25]:
save_model(model, 'markov', 'rnn')

In [9]:
def test_model(model, data_loader):
    model.eval()
    all_outputs = []
    all_targets = []
    with torch.no_grad():
        for sequences, targets in data_loader:
            sequences = sequences.float()
            targets = targets.float()
            lengths = torch.tensor([len(seq) for seq in sequences])
            
            # Forward pass
            outputs = model(sequences, lengths)
            all_outputs.append(outputs)
            all_targets.append(targets)
            
    return torch.cat(all_outputs), torch.cat(all_targets)

loaded_model = load_model('markov', 'rnn')

# Test the model on the training set
outputs, targets = test_model(loaded_model, train_loader)

# Convert outputs to probabilities
probabilities = torch.sigmoid(outputs)

# Convert probabilities to binary predictions
predictions = (probabilities > 0.5).float()

# Evaluate the predictions
accuracy = (predictions == targets.unsqueeze(1)).float().mean()
print(f'Accuracy on training set: {accuracy:.4f}')


Accuracy on training set: 0.9553


In [13]:
probabilities.unique

<bound method Tensor.unique of tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]])>

In [22]:
# model = RNNModel(input_size, hidden_size, num_layers, output_size)
# criterion = nn.BCEWithLogitsLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
# for batch_idx, (sequences, targets) in enumerate(train_loader):
#     print("Batch:", batch_idx)
#     print("Sequence shape:", sequences.shape)  # Print the shape of the input sequences
#     print("Targets shape:", targets.shape)  # Print the shape of the targets


In [24]:
# # Training loop (placeholder, implement training logic)
# for epoch in range(1):
#     for sequences, targets in train_loader:
#         lengths = [min(len(seq), sequence_length) for seq in sequences]
#         optimizer.zero_grad()
#         outputs = model(sequences, lengths)
#         loss = criterion(outputs, targets)
#         loss.backward()
#         optimizer.step()
